# AbbVie - Accumulations

In [16]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

In [17]:
# Read pdf into DataFrame
df = tabula.read_pdf("AbbVie_2017_Switzerland_EFPIA_Report_Final.pdf", pages='all', lattice=True)

In [18]:
df_export = df.copy()

column0 = df_export[df_export.columns[0]]

#Shift all where column[1] not empty
df_export[column0.notna()] = df_export[column0.notna()].shift(1, axis='columns')

#Drop Columns
df_export = df_export.drop(df.columns[9:14],axis=1)
df_export = df_export.drop(df.columns[0],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

#fill na
df_export = df_export.fillna("")

# Select
amount = df_export[df_export.type.str.contains('Aggregate amount')]
recipients = df_export[df_export.type.str.contains('Number of Recipients')]
df_export = pd.concat([amount, recipients])

#Numberize and sum
df_export = amounts_to_number(df_export)
df_export = sum_amounts(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"

export_acumulations(df_export, 'abbvie')